In [1]:
include("simulation.jl")

_ensemble! (generic function with 4 methods)

In [2]:
using LsqFit
using Plots

In [3]:
mutable struct SimulationData
    step::Int32
    clusters::Int32
    size::Vector{Int32}
    gyr::Vector{Float64}
end

mutable struct EnsembleData
    mrna_ang::Float64
    n_lip::Int32
    sim_data::Array{SimulationData}
end


In [6]:
d = load("../data/angular_coverage_test.jld");

┌ Warning: type Main.test_angular_coverage.EnsembleData not present in workspace; interpreting array as Array{Any}
└ @ JLD C:\Users\alenz\.julia\packages\JLD\S6t6A\src\JLD.jl:515
┌ Warning: type Main.test_angular_coverage.EnsembleData not present in workspace; reconstructing
└ @ JLD C:\Users\alenz\.julia\packages\JLD\S6t6A\src\jld_types.jl:697
┌ Warning: type Main.test_angular_coverage.SimulationData not present in workspace; interpreting array as Array{Any}
└ @ JLD C:\Users\alenz\.julia\packages\JLD\S6t6A\src\JLD.jl:515
┌ Warning: type Main.test_angular_coverage.SimulationData not present in workspace; reconstructing
└ @ JLD C:\Users\alenz\.julia\packages\JLD\S6t6A\src\jld_types.jl:697
┌ Warning: type Main.test_angular_coverage.SimulationData not present in workspace; interpreting array as Array{Any}
└ @ JLD C:\Users\alenz\.julia\packages\JLD\S6t6A\src\JLD.jl:515
┌ Warning: type Main.test_angular_coverage.SimulationData not present in workspace; interpreting array as Array{Any}
└ @ JL

In [7]:
angs = collect(0:10:360) .* pi ./ 180
n_Tests = length(angs)
n_lip = 125
iter = 20
skip = 43
n_steps = 1288 # 30 seconds in real time
dataset = d["data"];

In [8]:
step_vals = collect(1.0:skip:n_steps)
tot_lip = iter * n_lip
f_T(t, p) = p[1] .* t .+ 1
fit_ps = []
init_p = [rand()]

for test in 1:n_Tests
    av_cluster_size_at_step = similar(step_vals)
    for step_index in 1:length(step_vals)
        data_at_step = dataset[test].sim_data[step_index]
        av_cluster_size_at_step[step_index] = tot_lip/(tot_lip - sum(data_at_step.size) + data_at_step.clusters)
    end
    fit = curve_fit(f_T, step_vals, av_cluster_size_at_step, init_p)
    push!(fit_ps, fit.param)
end

In [13]:
plot(Int.(round.(angs .* 180 ./ pi)), reduce(hcat, fit_ps)'[:, 1], xlabel = "Angular Coverage (degrees)", ylabel = "Growth Rate", legend = false, c = :black, lw = 2, dpi = 300)
#savefig(joinpath(@__DIR__, "..\\figures\\growth_rate_vs_angular_coverage_blue.png"))

"c:\\Users\\alenz\\JuliaProjects\\liposome-aggregation\\figures\\growth_rate_vs_angular_coverage_blue.png"